# [희소행렬(sparse matrix)](https://radish-greens.tistory.com/1) 
### 희소행렬을 사용하는 이유
> 실전에서는 대규모 형렬을 다루어야 하는 경우가 흔합니다. 이 경우 메모리 문제가 생길 때 고려해볼 수 있는 것이 희소행렬입니다.   
> 예를 들어 추천 시스템을 만들기 위해 어떤 고객이 어떤 영화에 별점을 몇 개 주었는지를 나타내는 행렬을 이용하고자 합니다. 그런데 고객은 100만 명이 있고 영화 10만개가 있다고 하겠습니다. 그러면 100만 x 10만 크기의 행렬을 만드렁야 합니다. 원소 하나가 1바이트이라고 해도 1테라바이트 크기의 메모리가 필요합니다. 그런데 위 행렬은 원소가 대부분 0인 희소행렬일 것입니다. 사람 한명이 영화를 아무리 많이 봤자 1만개보다는 적을 것입니다. 그러면 나머지 9만 개 영화에 대해서는 행렬에 0으로 표현이됩니다.    
>  따라서 이 경우 개념적으로는 행렬로 생각하는 게 편한 경우라도, 실제 데이터는 다르게 표현하는 것이 좋습니다. 희소행렬을 표현하는 방식에는 coo, csr 등이 있습니다.

# 밀집행렬
> 물론 커다란 행렬이더라도 대부분의 원소가 0인 행렬이 아니라면, 데이터의 부담을 온전히 져야 하기 때문에 다른 알고리즘을 찾아야 할지도 모릅니다. 이 경우에는 희소행렬과 반대되는 개념으로 밀집행렬(dense matrix)라고 부릅니다.

## COO matrix

In [2]:
from scipy.sparse import coo_matrix

row = [0, 0, 0, 1, 2] # 행 인덱스를 담은 리스트
col = [0, 1, 2, 2, 3] # 열 인덱스를 담은 리스트
data = [2, 4, 2, 1, 5] # 원소 값을 담은 리스트

m = coo_matrix((data, (row, col)))

print(m.row, m.col, m.data)

[0 0 0 1 2] [0 1 2 2 3] [2 4 2 1 5]


## CSR matrix

In [3]:
from scipy.sparse import csr_matrix

indices = [0, 1, 2, 2, 3]
indptr = [0, 3, 4, 5]
data = [2, 4, 2, 1, 5]
m = csr_matrix((data, indices, indptr))

print(m.indices, m.indptr, m.data)

[0 1 2 2 3] [0 3 4 5] [2 4 2 1 5]


## DOK 방식 
> dock(Dictionary of keys)방식을 소개하겠습니다. dok는 좌표가 key이고 원소 값이 value인 딕셔너리 구조입니다. dok 방식은 희소행렬을 점진적으로 구축할 때 사용하기 좋습니다. 

In [12]:
from scipy.sparse import dok_matrix

m = dok_matrix((3, 4)) # 값이 0으로 채워진 (3, 4) 크기의 행렬을 만듭니다.
m[0, 1] = 4
m[2, 3] = 9
m.toarray()

array([[0., 4., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 9.]])

In [13]:
m.keys()

dict_keys([(0, 1), (2, 3)])

# 표현 방식 간 간단한 비교
- 희소행렬을 나타내는 각각의 방식에는 장단점이 있습니다. 어떤 방식은 다른 방식보다 특정 동작을 수행할 때 더 빠릅니다.  
- 우선 `coo`는 희소행렬을 생성할 때 좀 더 직관적입니다. 그렇기 때문에 우선 coo 행렬을 만들고 나서, tocsr이나 tocsc 메서드를 통해 다른 형식의 희소행렬로 바꾸어서 작업해주면 됩니다.  
- `csr 방식과 csc 방식`은 산술연산과 행렬*벡터 곱 연산을 빠르게 수행합니다. 그래서 대부분 이 두 방식을 많이 쓰는 것 같습니다. 희소행렬을 이용한 연산에서는 0을 곱하는 연산을 하지 않아도 된다는 장점도 있습니다.  
- 반면 `dock` 방식은 행렬 연산이 느리다고 합니다. 그러나 특정 좌표에 있는 값에 빠르게 접근할 수 있기 때문에 희소행렬을 점진적으로 구축할 때 이용하기 좋고, `coo` 방식으로의 변환이 빠릅니다.